# Import thư viện

In [340]:
import numpy as np 
import pandas as pd
from itables import show    
from scipy.stats import skew,boxcox_normmax, zscore
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, FunctionTransformer, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import itables.options as opt
import os
opt.maxBytes = 5000000

# Nạp dữ liệu

In [341]:
train = pd.read_csv('../data/train.csv', encoding='utf-8')
test = pd.read_csv('../data/test.csv', encoding='utf-8')

# Tiền xử lý dữ liệu Name Artist


## Chỉ giữ lại Artist và class
+ Sửa lỗi ngôn ngữ
+ Dọn dẹp các tên được xem là corrupted (Chứa các kí tự đặt biệt mà ko có các chữ cái)
+ Cắt chuỗi nhóm tên thành từng tên độc lập

In [342]:
import re
#chỉ có chữ
tmp = train.copy()
tmp = tmp[['Id','Artist Name']]
tmp = tmp.drop_duplicates()
show(tmp)

#loại bỏ chữ cái alphabet
not_alphabet = tmp[~tmp['Artist Name'].str.contains(r'(?=.*[A-Za-z])')]
show(not_alphabet)

#có số là được
num_included = tmp[tmp['Artist Name'].str.contains(r'\d')]
show(num_included)

#chỉ có số
num_only = tmp[tmp['Artist Name'].str.isdigit()]
show(num_only)

#gồm số và chữ
num_alphabelt = tmp[tmp['Artist Name'].str.contains(r'(?=.*[A-Za-z])(?=.*\d)')]
show(num_alphabelt)


Loading ITables v2.5.2 from the internet... (need help?)


Loading ITables v2.5.2 from the internet... (need help?)


Loading ITables v2.5.2 from the internet... (need help?)


Loading ITables v2.5.2 from the internet... (need help?)


Loading ITables v2.5.2 from the internet... (need help?)


In [343]:
diff_rows = not_alphabet[~not_alphabet['Artist Name'].isin(num_only['Artist Name'])]
show(diff_rows)

Loading ITables v2.5.2 from the internet... (need help?)


In [344]:
diff_rows = test[test['Artist Name'].isin(diff_rows['Artist Name'])]
show(diff_rows)

Loading ITables v2.5.2 from the internet... (need help?)


## Dựa trên dữ liệu đã bị corrupted trên cả test train ta có thể coi nó như cùng chung format 
**phương hướng xử lí:**
- Trim
- Regex chuỗi gán label đối với nhóm ca sĩ, đồng thời giữ chuỗi gốc đã có nhãn làm tập train.
- Thiết lập quy luật cho input là nếu vocab đó (token) không có trong train thì mặc định embedding là vector 0.

In [345]:
import re
import pandas as pd



def explode_artists(df):
    rows = []
    for idx, row in df.iterrows():
        artists = [x.strip() for x in str(row["Artist Name"]).split(",")]
        for artist in artists:
            rows.append({
                "Original Index": idx,           # to regroup later
                "Artist Name": artist,
                "Class": row.get("Class", None)
            })
    return pd.DataFrame(rows)


In [346]:
train_exp = train.copy()
test_exp  = test.copy()
# =======================================================
# EXPLODE
# =======================================================
train_exp = explode_artists(train_exp)
test_exp  = explode_artists(test_exp)

# =======================================================
# GROUP BACK 
# =======================================================

def group_back(df_exp):
    return (
        df_exp
        .groupby("Original Index")
        .agg({
            "Artist Name": lambda x: ", ".join(sorted(set(x))),  
            "Class": "first"
        })
        .reset_index()
    )

train_clean_grouped = pd.concat([train_exp,group_back(train_exp)],ignore_index=True)

train_clean_grouped = train_clean_grouped.drop(columns=["Original Index"])

train_clean_grouped = train_clean_grouped.drop_duplicates().reset_index(drop=True)

show(train_clean_grouped)


Loading ITables v2.5.2 from the internet... (need help?)


# Lưu lại dữ liệu

In [347]:
exp_dir = "../exps"
if os.path.exists(exp_dir) == False:
    os.makedirs(exp_dir,exist_ok=True)

save_dir = f"{exp_dir}/Preproccessed"
os.makedirs(save_dir,exist_ok=True)

In [348]:
train_clean_grouped.to_csv(f'{save_dir}/exp2_NamesLabeling_Train.csv',index=False)
train_exp = train_exp.drop(columns=["Original Index"])
train_exp = train_exp.drop_duplicates().reset_index(drop=True)
train_exp.to_csv(f'{save_dir}/exp2_vocab.csv',index=False)